<a href="https://colab.research.google.com/github/kri8tinam/Low-Resource-Deontic-Modality-Classification-in-EU-Legislation/blob/main/LEGALBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
#loading and visualizing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import string
import torch
import os
import random as rn

#preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#tokenization
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize


!pip install transformers
from transformers import AutoTokenizer, AutoModel

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
from tensorflow.keras import callbacks

!pip install keras_lr_finder
!pip install torch_optimizer

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from keras_lr_finder import LRFinder
# from clr_callback import CyclicLR


!pip install tensorflow-model-optimization
import tensorflow_model_optimization as tfmot

!pip install keras-tuner
import keras_tuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

from tensorflow.keras import regularizers
import time

# evaluation
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.0 MB/s eta 0:00:00
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.9 MB/s eta 0:00:00


<ipython-input-1-0d7e7f60ed9c>:53: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Data Loading

In [3]:
train_data = pd.read_csv('/content/drive/MyDrive/TRAIN.csv', delimiter= ',')
test_data = pd.read_csv('/content/drive/MyDrive/TEST.csv', delimiter= ',')

# Samples
train100 = pd.read_csv('/content/drive/MyDrive/train100.csv', delimiter= ',')
train200 = pd.read_csv('/content/drive/MyDrive/train200.csv', delimiter= ',')
train300 = pd.read_csv('/content/drive/MyDrive/train300.csv', delimiter= ',')
train350 = pd.read_csv('/content/drive/MyDrive/train350.csv', delimiter= ',')
train400 = pd.read_csv('/content/drive/MyDrive/train400.csv', delimiter= ',')
train450 = pd.read_csv('/content/drive/MyDrive/train450.csv', delimiter= ',')

# Filtered & Augmented
filtered_150 = pd.read_csv('/content/drive/MyDrive/train_filtered_150.csv', delimiter= ',')
filtered_312 = pd.read_csv('/content/drive/MyDrive/train_filtered_312.csv', delimiter= ',')
aug_train150 = pd.read_csv('/content/drive/MyDrive/aug_train150.csv', delimiter= ',')
aug_train312 = pd.read_csv('/content/drive/MyDrive/aug_train312.csv', delimiter= ',')

In [4]:
test_data

,text,category
0,Organizations must ensure that their product r...,obligations
1,For a period of thirty years from 1 October 20...,right
2,Member States may adopt measures to limit acce...,power
3,For a period of seventeen years from 1 October...,right
4,Organizations must ensure that their product r...,obligations
...,...,...
321,Member States may adopt measures to limit acce...,power
322,Organizations must ensure that their product a...,obligations
323,The Union shall ensure the right to protection...,prohibitions
324,For a period of twenty years from 1 November 2...,power


In [5]:
print(train_data['category'].value_counts())
print(test_data['category'].value_counts())

permission     117
prohibition    108
obligation      85
omission        84
power           74
right           47
Name: category, dtype: int64
obligations     71
permission      66
right           60
prohibitions    58
power           54
omission        17
Name: category, dtype: int64


# Data Preparation

In [6]:
char_list = [r'\d', '\+', ':', '\\n', '#', '$', '%', '&','"', ',', '-',
             ';', '<', '=', '>', '@', '!', '^', '_', '`', '{', '|', '}', '~']   # list of special characters/punctuation

def char_search(text, col, characters):
  for char in characters:
    count = text[col].str.contains(char).sum()
    print(f'{char} special character is present in {count} entries')

char_search(test_data, 'text', char_list)

\d special character is present in 130 entries
\+ special character is present in 0 entries
: special character is present in 0 entries
\n special character is present in 0 entries
# special character is present in 0 entries
$ special character is present in 326 entries
% special character is present in 0 entries
& special character is present in 0 entries
" special character is present in 0 entries
, special character is present in 129 entries
- special character is present in 32 entries
; special character is present in 0 entries
< special character is present in 0 entries
= special character is present in 0 entries
> special character is present in 0 entries
@ special character is present in 0 entries
! special character is present in 0 entries
^ special character is present in 326 entries
_ special character is present in 0 entries
` special character is present in 0 entries
{ special character is present in 0 entries
| special character is present in 326 entries
} special characte

In [7]:
def text_cleanse(text, col, characters):
  text[col] = text[col].str.lower() # convert to lowercase
  for char in characters:
    text[col] = text[col].str.replace(char, '', regex = True) # remove special characters

text_cleanse(test_data, 'text', char_list)

char_search(test_data, 'text', char_list)

\d special character is present in 0 entries
\+ special character is present in 0 entries
: special character is present in 0 entries
\n special character is present in 0 entries
# special character is present in 0 entries
$ special character is present in 326 entries
% special character is present in 0 entries
& special character is present in 0 entries
" special character is present in 0 entries
, special character is present in 0 entries
- special character is present in 0 entries
; special character is present in 0 entries
< special character is present in 0 entries
= special character is present in 0 entries
> special character is present in 0 entries
@ special character is present in 0 entries
! special character is present in 0 entries
^ special character is present in 326 entries
_ special character is present in 0 entries
` special character is present in 0 entries
{ special character is present in 0 entries
| special character is present in 326 entries
} special character is 

In [8]:
# CLEANING THE DATASETS

def cleaning(text):
  text = text.replace('$', '')
  text = text.replace('^', '')
  text = re.sub(r'\n\n', '. ', text)
  text = re.sub(r"\(\d+\)", "", text)
  text = text.replace('   ', " ")
  text = re.sub(r"\b\d\. \b", "", text)
  text = text.strip()

  return text

## Label Encoding

In [9]:
le = LabelEncoder()
train_data['category'] = le.fit_transform(train_data["category"])
test_data['category'] = le.fit_transform(test_data["category"])

filtered_150['category'] = le.fit_transform(filtered_150["category"])
filtered_312['category'] = le.fit_transform(filtered_312["category"])
aug_train150['category'] = le.fit_transform(aug_train150["category"])
aug_train312['category'] = le.fit_transform(aug_train312["category"])

train100['category'] = le.fit_transform(train100["category"])
train200['category'] = le.fit_transform(train200["category"])
train300['category'] = le.fit_transform(train300["category"])
train350['category'] = le.fit_transform(train350["category"])
train400['category'] = le.fit_transform(train400["category"])
train450['category'] = le.fit_transform(train450["category"])

# LegalBERT

## Tokenization

In [10]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

def tokenization(tokenizer, dataset, col):
  dataset['tokens'] = dataset[col].apply(lambda x: tokenizer.tokenize(x))
  dataset['tokens'] = dataset['tokens'].apply(lambda x: ['[CLS]'] + x + ['[SEP]'])
  dataset['token count'] = dataset['tokens'].apply(lambda x: len(x))

  return dataset

train_data = tokenization(tokenizer, train_data, 'text')
test_data = tokenization(tokenizer, test_data, 'text')



In [11]:
num_tokens = list(train_data["token count"])
print('Max number of tokens in train data', max(num_tokens))

num_tokens1 = list(test_data["token count"])
print('Max number of tokens in test data:', max(num_tokens1))

Max number of tokens in train data 42
Max number of tokens in test data: 57


In [12]:
# PADDING
def padding(tokens, count):
  if count < 100:
    num_added_paddings = 100 - count
    for i in range(num_added_paddings):
      tokens = tokens + ['[PAD]']
  return tokens

# CONVERT TO TOKEN IDS
def token_ids(padding, dataset, col_tokens, col_count):
  dataset[col_tokens] = dataset.apply(lambda row: padding(row[col_tokens], row[col_count]), axis=1)
  dataset['token ids'] = dataset[col_tokens].apply(lambda x: tokenizer.convert_tokens_to_ids(x))
  dataset['token ids'] = dataset['token ids'].apply(lambda x: torch.tensor(x).unsqueeze(0))

  return dataset

train_data = token_ids(padding, train_data, 'tokens', 'token count')
test_data = token_ids(padding, test_data, 'tokens', 'token count')


In [13]:
# ATTENTION MASK

def attention(tokens):
  attention_mask = [1 if i!= '[PAD]' else 0 for i in tokens]
  attention_mask = torch.tensor(attention_mask).unsqueeze(0)
  return attention_mask

train_data['attention mask'] = train_data['tokens'].apply(lambda x: attention(x))
test_data['attention mask'] = test_data['tokens'].apply(lambda x: attention(x))

test_data

,text,category,tokens,token count,token ids,attention mask
0,organizations must ensure that their product r...,0,"[[CLS], organizations, must, ensure, that, the...",19,"[[tensor(101), tensor(4072), tensor(311), tens...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
1,for a period of thirty years from october me...,5,"[[CLS], for, a, period, of, thirty, years, fro...",51,"[[tensor(101), tensor(217), tensor(145), tenso...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
2,member states may adopt measures to limit acce...,3,"[[CLS], member, states, may, adopt, measures, ...",29,"[[tensor(101), tensor(262), tensor(265), tenso...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
3,for a period of seventeen years from october ...,5,"[[CLS], for, a, period, of, seventeen, years, ...",51,"[[tensor(101), tensor(217), tensor(145), tenso...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
4,organizations must ensure that their product r...,0,"[[CLS], organizations, must, ensure, that, the...",17,"[[tensor(101), tensor(4072), tensor(311), tens...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
...,...,...,...,...,...,...
321,member states may adopt measures to limit acce...,3,"[[CLS], member, states, may, adopt, measures, ...",29,"[[tensor(101), tensor(262), tensor(265), tenso...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
322,organizations must ensure that their product a...,0,"[[CLS], organizations, must, ensure, that, the...",17,"[[tensor(101), tensor(4072), tensor(311), tens...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
323,the union shall ensure the right to protection...,4,"[[CLS], the, union, shall, ensure, the, right,...",19,"[[tensor(101), tensor(207), tensor(403), tenso...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
324,for a period of twenty years from november m...,3,"[[CLS], for, a, period, of, twenty, years, fro...",51,"[[tensor(101), tensor(217), tensor(145), tenso...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."


## Train/Test split

In [14]:
# Adjust the tensor shape to fit the BERT model - has to be:
# [batch_size, sequence_length = max_tokens]

def split_and_format_batches(df):
  ids = df['token ids']
  att = df['attention mask']
  y = df['category']

  ids = torch.stack(tuple(ids)).squeeze(1)
  att = torch.stack(tuple(att)).squeeze(1)
  y = torch.Tensor(y.values)

  return ids, att, y

X_train_ids, X_train_att, y_train = split_and_format_batches(train_data)
X_test_ids, X_test_att, y_test = split_and_format_batches(test_data)


In [15]:
print(X_train_ids.shape, X_test_att.shape)

torch.Size([515, 100]) torch.Size([326, 100])


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


**Batches**

---



In [17]:
# Define the batches and data loader
from torch.utils.data import DataLoader


# Whole train dataset
dataset = torch.utils.data.TensorDataset(X_train_ids, X_train_att, y_train) # contains all tensors for the ids, attention masks and labels
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)      # divides the dataset into batches of # samples (ids, att, labels)

# Test
test_dataset = torch.utils.data.TensorDataset(X_test_ids, X_test_att, y_test)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16)


## Models and Train/Evaluation function definition

**Model deifinition**

---



In [18]:
from torch import nn

# BASELINE MODEL
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.linear(self.dropout(pooled_output))
        return logits


In [19]:
# MODIFIED - DROPOUT RATIO = 0.2

class BERTClassifier1(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier1, self).__init__()
        self.bert = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.linear(self.dropout(pooled_output))
        return logits

In [20]:
# MODIFIED - 2 LIN LAYERS + RELU + 0.2 DROPOUT, FINAL LAYER - SOFTMAX

class BERTClassifier2(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier2, self).__init__()
        self.bert = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

        # 1ST LAYER
        self.linear1 = nn.Linear(768, 512)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)

        # 2ND LAYER
        self.linear2 = nn.Linear(512, 256)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        # FINAL LAYER
        self.output_layer = nn.Linear(256, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.relu(self.linear1(pooled_output))
        x = self.dropout1(x)
        x = self.relu(self.linear2(x))
        x = self.dropout2(x)
        logits = self.output_layer(x)
        return logits

**Training loop**

---



In [21]:
torch.cuda.empty_cache()     # Clearing Cache space for fresh Model run
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'0.0M'

In [22]:
# Initialize the model's weights with random values
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

In [23]:
from torch.optim import Adam

def train(model, initialize_weights, train_data, data_loader, learning_rate, epochs):

    # Optional: You can specify a random seed for reproducibility
    seed_value = 42
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.apply(initialize_weights)   # initialize random weights


    loss_alg = nn.CrossEntropyLoss() #.cuda()
    optimizer = Adam(model.parameters(), lr= learning_rate)
    model = model.cuda()


    for epoch in range(epochs):
      total_acc_train = 0
      total_loss_train = 0
      model.train()

      for batch in data_loader:                # batch = list of:
        input_id = batch[0].to(device)         # [batch_size, 512]
        mask = batch[1].to(device)             # [batch_size, 512]
        label = batch[2].long().to(device)     # [batch_size]

        optimizer.zero_grad()

        # FORWARD PASS

        output = model(input_id, mask) # output = tensor of [batch size, 2] with the probabilities for each of the 2 classes

        batch_loss = loss_alg(output, label)   # calculate the loss between the predicted and actual label per batch
        total_loss_train += batch_loss.item()  # accumulate the loss

        acc = (output.argmax(dim=1) == label).sum().item()  # calculate the accuracy
        total_acc_train += acc

        # BACKWARD PASS
        batch_loss.backward()
        optimizer.step()        # update the gradient with the given learning rate

      print(f'Epochs: {epoch + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
      | Train Accuracy: {total_acc_train / len(train_data): .3f}')


**Evaluation function**

---

In [24]:
# Evaluation

def evaluation(model, test_dataloader, test_dataset):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = model.cuda()

  total_acc_test = 0
  total_predictions = []
  model.eval()

  with torch.no_grad():
    for batch in test_dataloader:
      input_id = batch[0].to(device)         # [batch_size, 512]
      mask = batch[1].to(device)             # [batch_size, 512]
      label = batch[2].long().to(device)     # [batch_size]

      output = model(input_id, mask)

      y_pred = output.argmax(dim=1)

      acc = (output.argmax(dim=1) == label).sum().item()  # calculate the accuracy
      total_acc_test += acc

      total_predictions.extend(y_pred.cpu().numpy()) # accumulate the predicted labels from each iteration (batch)


  print(f'Test Accuracy: {total_acc_test / len(test_dataset): .3f}')

  return total_predictions



## Benchmark

In [25]:
num_classes = 6  # Specify the number of output classes

# WHOLE dataset
model = BERTClassifier(num_classes)
print('Model on WHOLE dataset')
train(model,initialize_weights, dataset, dataloader, learning_rate = 2e-5, epochs = 3)

print('Evaluation on the test data (326 samples)')
total_pred = evaluation(model, test_dataloader, test_dataset)

Model on WHOLE dataset
Epochs: 1 | Train Loss:  0.065       | Train Accuracy:  0.645
Epochs: 2 | Train Loss:  0.007       | Train Accuracy:  0.973
Epochs: 3 | Train Loss:  0.005       | Train Accuracy:  0.975
Evaluation on the test data (326 samples)
Test Accuracy:  0.595


## Minimal Sample

In [26]:
# Tokenizing the samples
train100 = tokenization(tokenizer, train100, 'text')
train200 = tokenization(tokenizer, train200, 'text')
train300 = tokenization(tokenizer, train300, 'text')
train350 = tokenization(tokenizer, train350, 'text')
train400 = tokenization(tokenizer, train400, 'text')
train450 = tokenization(tokenizer, train450, 'text')

# Padding
train100 = token_ids(padding, train100, 'tokens', 'token count')
train200 = token_ids(padding, train200, 'tokens', 'token count')
train300 = token_ids(padding, train300, 'tokens', 'token count')
train350 = token_ids(padding, train350, 'tokens', 'token count')
train400 = token_ids(padding, train400, 'tokens', 'token count')
train450 = token_ids(padding, train450, 'tokens', 'token count')

# Attention
train100['attention mask'] = train100['tokens'].apply(lambda x: attention(x))
train200['attention mask'] = train200['tokens'].apply(lambda x: attention(x))
train300['attention mask'] = train300['tokens'].apply(lambda x: attention(x))
train350['attention mask'] = train350['tokens'].apply(lambda x: attention(x))
train400['attention mask'] = train400['tokens'].apply(lambda x: attention(x))
train450['attention mask'] = train450['tokens'].apply(lambda x: attention(x))

# Formatting
train100_ids, train100_att, y_train100 = split_and_format_batches(train100)
train200_ids, train200_att, y_train200 = split_and_format_batches(train200)
train300_ids, train300_att, y_train300 = split_and_format_batches(train300)
train350_ids, train350_att, y_train350 = split_and_format_batches(train350)
train400_ids, train400_att, y_train400 = split_and_format_batches(train400)
train450_ids, train450_att, y_train450 = split_and_format_batches(train450)

# Batches
batch_size = 4 # adjust to test for different batch sizes

# Whole train data
dataset = torch.utils.data.TensorDataset(X_train_ids, X_train_att, y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Test data
test_dataset = torch.utils.data.TensorDataset(X_test_ids, X_test_att, y_test)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

dataset100 = torch.utils.data.TensorDataset(train100_ids, train100_att, y_train100)
dataloader100 = DataLoader(dataset100, batch_size=batch_size, shuffle=True)

dataset200 = torch.utils.data.TensorDataset(train200_ids, train200_att, y_train200)
dataloader200 = DataLoader(dataset200, batch_size=batch_size, shuffle=True)

dataset300 = torch.utils.data.TensorDataset(train300_ids, train300_att, y_train300)
dataloader300 = DataLoader(dataset300, batch_size=batch_size, shuffle=True)

dataset350 = torch.utils.data.TensorDataset(train350_ids, train350_att, y_train350)
dataloader350 = DataLoader(dataset350, batch_size=batch_size, shuffle=True)

dataset400 = torch.utils.data.TensorDataset(train400_ids, train400_att, y_train400)
dataloader400 = DataLoader(dataset400, batch_size=batch_size, shuffle=True)

dataset450 = torch.utils.data.TensorDataset(train450_ids, train450_att, y_train450)
dataloader450 = DataLoader(dataset450, batch_size=batch_size, shuffle=True)

In [27]:
# TRAINING

# 100 samples
model100 = BERTClassifier(num_classes)
print('Model on 100 SAMPLES')
torch.cuda.empty_cache()     # Clearing Cache space for fresh Model run
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'
train(model100,initialize_weights, dataset100, dataloader100, learning_rate = 2e-5, epochs = 3)

# 200 samples
model200 = BERTClassifier(num_classes)
print('Model on 200 SAMPLES')
torch.cuda.empty_cache()
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'
train(model200,initialize_weights, dataset200, dataloader200,  learning_rate = 2e-5, epochs = 3)

# 300 samples
model300 = BERTClassifier(num_classes)
torch.cuda.empty_cache()
print('MODEL ON 300 SAMPLES')
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'
train(model300,initialize_weights, dataset300, dataloader300, learning_rate = 2e-5, epochs = 3)

# 350 samples
model350 = BERTClassifier(num_classes)
print('MODEL ON 350 SAMPLES')
torch.cuda.empty_cache()
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'
train(model350,initialize_weights, dataset350, dataloader350, learning_rate = 2e-5, epochs = 3)

# 400 samples
model400 = BERTClassifier(num_classes)
print('MODEL ON 400 SAMPLES')
torch.cuda.empty_cache()
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'
train(model400,initialize_weights, dataset400, dataloader400, learning_rate = 2e-5, epochs = 3)

# 450 samples
model450 = BERTClassifier(num_classes)
print('MODEL ON 450 SAMPLES')
torch.cuda.empty_cache()
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'
train(model450,initialize_weights, dataset450, dataloader450, learning_rate = 2e-5, epochs = 3)


Model on 100 SAMPLES
Epochs: 1 | Train Loss:  0.509       | Train Accuracy:  0.200
Epochs: 2 | Train Loss:  0.476       | Train Accuracy:  0.260
Epochs: 3 | Train Loss:  0.241       | Train Accuracy:  0.640
Model on 200 SAMPLES
Epochs: 1 | Train Loss:  0.433       | Train Accuracy:  0.375
Epochs: 2 | Train Loss:  0.085       | Train Accuracy:  0.905
Epochs: 3 | Train Loss:  0.051       | Train Accuracy:  0.945
MODEL ON 300 SAMPLES
Epochs: 1 | Train Loss:  0.348       | Train Accuracy:  0.447
Epochs: 2 | Train Loss:  0.082       | Train Accuracy:  0.897
Epochs: 3 | Train Loss:  0.062       | Train Accuracy:  0.930
MODEL ON 350 SAMPLES
Epochs: 1 | Train Loss:  0.267       | Train Accuracy:  0.606
Epochs: 2 | Train Loss:  0.073       | Train Accuracy:  0.903
Epochs: 3 | Train Loss:  0.023       | Train Accuracy:  0.971
MODEL ON 400 SAMPLES
Epochs: 1 | Train Loss:  0.248       | Train Accuracy:  0.605
Epochs: 2 | Train Loss:  0.049       | Train Accuracy:  0.955
Epochs: 3 | Train Loss:  0.

In [28]:
pred = evaluation(model, test_dataloader, test_dataset )
pred100 = evaluation(model100, test_dataloader, test_dataset)
pred200 = evaluation(model200, test_dataloader, test_dataset)
pred300 = evaluation(model300, test_dataloader, test_dataset)
pred350 = evaluation(model350, test_dataloader, test_dataset)
pred400 = evaluation(model400, test_dataloader, test_dataset)
pred450 = evaluation(model450, test_dataloader, test_dataset)

Test Accuracy:  0.595
Test Accuracy:  0.540
Test Accuracy:  0.448
Test Accuracy:  0.460
Test Accuracy:  0.472
Test Accuracy:  0.417
Test Accuracy:  0.279


## Similarity filtering and Data augmentation

In [29]:
# Filtered data

filtered_150 = pd.read_csv('/content/drive/MyDrive/train_filtered_150.csv', delimiter= ',')
filtered_312 = pd.read_csv('/content/drive/MyDrive/train_filtered_312.csv', delimiter= ',')

filtered_150['category'] = le.fit_transform(filtered_150["category"])
filtered_312['category'] = le.fit_transform(filtered_312["category"])

filtered_150 = tokenization(tokenizer, filtered_150, 'text')
filtered_312 = tokenization(tokenizer, filtered_312, 'text')

filtered_150 = token_ids(padding, filtered_150, 'tokens', 'token count')
filtered_312 = token_ids(padding, filtered_312, 'tokens', 'token count')

filtered_150['attention mask'] = filtered_150['tokens'].apply(lambda x: attention(x))
filtered_312['attention mask'] = filtered_312['tokens'].apply(lambda x: attention(x))

ids150, att150, y150 = split_and_format_batches(filtered_150)
ids312, att312, y312 = split_and_format_batches(filtered_312)

dataset_150 = torch.utils.data.TensorDataset(ids150, att150, y150)
dataloader_150= DataLoader(dataset_150, batch_size=4, shuffle=True)

dataset_312 = torch.utils.data.TensorDataset(ids312, att312, y312)
dataloader_312 = DataLoader(dataset_312, batch_size=4, shuffle=True)

In [30]:
aug_train150 = pd.read_csv('/content/drive/MyDrive/aug_train150.csv', delimiter= ',')
aug_train312 = pd.read_csv('/content/drive/MyDrive/aug_train312.csv', delimiter= ',')


aug_train150['category'] = le.fit_transform(aug_train150["category"])
aug_train312['category'] = le.fit_transform(aug_train312["category"])

aug_train150 = tokenization(tokenizer, aug_train150, 'text')
aug_train312 = tokenization(tokenizer, aug_train312, 'text')

aug_train150 = token_ids(padding, aug_train150, 'tokens', 'token count')
aug_train312 = token_ids(padding, aug_train312, 'tokens', 'token count')

aug_train150['attention mask'] = aug_train150['tokens'].apply(lambda x: attention(x))
aug_train312['attention mask'] = aug_train312['tokens'].apply(lambda x: attention(x))

aug_ids150, aug_att150, aug_y150 = split_and_format_batches(aug_train150)
aug_ids312, aug_att312, aug_y312 = split_and_format_batches(aug_train312)

aug_dataset150 = torch.utils.data.TensorDataset(aug_ids150, aug_att150, aug_y150)
aug_dataloader150 = DataLoader(aug_dataset150, batch_size=4, shuffle=True)

aug_dataset312 = torch.utils.data.TensorDataset(aug_ids312, aug_att312, aug_y312)
aug_dataloader312 = DataLoader(aug_dataset312, batch_size=4, shuffle=True)


In [31]:
# 150 filtered
model = BERTClassifier(num_classes)
print('Model on 150 filtered dataset')
train(model,initialize_weights, dataset_150, dataloader_150, learning_rate = 2e-5, epochs = 3)

print('Evaluation on the test data (326 samples)')
total_pred = evaluation(model, test_dataloader, test_dataset)

Model on 150 filtered dataset
Epochs: 1 | Train Loss:  0.316       | Train Accuracy:  0.560
Epochs: 2 | Train Loss:  0.179       | Train Accuracy:  0.740
Epochs: 3 | Train Loss:  0.074       | Train Accuracy:  0.913
Evaluation on the test data (326 samples)
Test Accuracy:  0.371


In [32]:
# 312 filtered
model = BERTClassifier(num_classes)
print('Model on 312 filtered dataset')
train(model,initialize_weights, dataset_312, dataloader_312, learning_rate = 2e-5, epochs = 3)

print('Evaluation on the test data (326 samples)')
total_pred = evaluation(model, test_dataloader, test_dataset)

Model on 312 filtered dataset
Epochs: 1 | Train Loss:  0.302       | Train Accuracy:  0.574
Epochs: 2 | Train Loss:  0.061       | Train Accuracy:  0.910
Epochs: 3 | Train Loss:  0.038       | Train Accuracy:  0.962
Evaluation on the test data (326 samples)
Test Accuracy:  0.393


In [33]:
# 150 augmented
model = BERTClassifier(num_classes)
print('Model on 150 augmented dataset')
train(model,initialize_weights, aug_dataset150, aug_dataloader150, learning_rate = 2e-5, epochs = 3)

print('Evaluation on the test data (326 samples)')
total_pred = evaluation(model, test_dataloader, test_dataset)

Model on 150 augmented dataset
Epochs: 1 | Train Loss:  0.436       | Train Accuracy:  0.392
Epochs: 2 | Train Loss:  0.119       | Train Accuracy:  0.828
Epochs: 3 | Train Loss:  0.055       | Train Accuracy:  0.923
Evaluation on the test data (326 samples)
Test Accuracy:  0.552


In [34]:
# 312 augmented
model = BERTClassifier(num_classes)
print('Model on 312 augmented dataset')
train(model,initialize_weights, aug_dataset312, aug_dataloader312, learning_rate = 2e-5, epochs = 3)

print('Evaluation on the test data (326 samples)')
total_pred = evaluation(model, test_dataloader, test_dataset)

Model on 312 augmented dataset
Epochs: 1 | Train Loss:  0.218       | Train Accuracy:  0.674
Epochs: 2 | Train Loss:  0.037       | Train Accuracy:  0.948
Epochs: 3 | Train Loss:  0.024       | Train Accuracy:  0.974
Evaluation on the test data (326 samples)
Test Accuracy:  0.540


## Parameter fine-tuning

BASELINE

In [35]:
num_classes = 6  # Specify the number of output classes
learning_rates = (1e-5, 2e-5, 3e-5, 4e-5, 5e-5)
batch_sizes = (4,8,16,32)
epochs = (3,4)

for epoch in epochs:
  print('\n')
  print('Epochs =', epoch)
  for batch in batch_sizes:
    print('\nFor a batch size of',batch)
    dataset = torch.utils.data.TensorDataset(X_train_ids, X_train_att, y_train)
    dataloader = DataLoader(dataset, batch_size=batch, shuffle=True)
    for lr in learning_rates:
      print('\n')
      model = BERTClassifier(num_classes)
      print('Training the model')
      train(model,initialize_weights, dataset, dataloader, lr, epoch)
      total_pred = evaluation(model, test_dataloader, test_dataset)



Epochs = 3

For a batch size of 4


Training the model
Epochs: 1 | Train Loss:  0.211       | Train Accuracy:  0.709
Epochs: 2 | Train Loss:  0.039       | Train Accuracy:  0.955
Epochs: 3 | Train Loss:  0.011       | Train Accuracy:  0.984
Test Accuracy:  0.525


Training the model
Epochs: 1 | Train Loss:  0.234       | Train Accuracy:  0.664
Epochs: 2 | Train Loss:  0.063       | Train Accuracy:  0.934
Epochs: 3 | Train Loss:  0.021       | Train Accuracy:  0.977
Test Accuracy:  0.534


Training the model
Epochs: 1 | Train Loss:  0.325       | Train Accuracy:  0.526
Epochs: 2 | Train Loss:  0.064       | Train Accuracy:  0.930
Epochs: 3 | Train Loss:  0.062       | Train Accuracy:  0.934
Test Accuracy:  0.512


Training the model
Epochs: 1 | Train Loss:  0.491       | Train Accuracy:  0.280
Epochs: 2 | Train Loss:  0.204       | Train Accuracy:  0.709
Epochs: 3 | Train Loss:  0.110       | Train Accuracy:  0.862
Test Accuracy:  0.408


Training the model
Epochs: 1 | Train Loss:  0.

DROPOUT = 0.2

In [36]:
num_classes = 6  # Specify the number of output classes
learning_rates = (1e-5, 2e-5, 3e-5, 4e-5, 5e-5)
batch_sizes = (4,8,16,32)
epochs = (3,4)


for epoch in epochs:
  print('\n')
  print('Epochs =', epoch)
  for batch in batch_sizes:
    print('\nFor a batch size of',batch)
    dataset = torch.utils.data.TensorDataset(X_train_ids, X_train_att, y_train)
    dataloader = DataLoader(dataset, batch_size=batch, shuffle=True)
    for lr in learning_rates:
      print('\n')
      model = BERTClassifier1(num_classes)
      print('Training the model')
      train(model,initialize_weights, dataset, dataloader, lr, epoch)
      total_pred = evaluation(model, test_dataloader, test_dataset)



Epochs = 3

For a batch size of 4


Training the model
Epochs: 1 | Train Loss:  0.218       | Train Accuracy:  0.695
Epochs: 2 | Train Loss:  0.038       | Train Accuracy:  0.953
Epochs: 3 | Train Loss:  0.014       | Train Accuracy:  0.990
Test Accuracy:  0.540


Training the model
Epochs: 1 | Train Loss:  0.242       | Train Accuracy:  0.650
Epochs: 2 | Train Loss:  0.049       | Train Accuracy:  0.946
Epochs: 3 | Train Loss:  0.028       | Train Accuracy:  0.967
Test Accuracy:  0.589


Training the model
Epochs: 1 | Train Loss:  0.296       | Train Accuracy:  0.573
Epochs: 2 | Train Loss:  0.079       | Train Accuracy:  0.909
Epochs: 3 | Train Loss:  0.040       | Train Accuracy:  0.967
Test Accuracy:  0.521


Training the model
Epochs: 1 | Train Loss:  0.425       | Train Accuracy:  0.336
Epochs: 2 | Train Loss:  0.154       | Train Accuracy:  0.806
Epochs: 3 | Train Loss:  0.166       | Train Accuracy:  0.798
Test Accuracy:  0.350


Training the model
Epochs: 1 | Train Loss:  0.

In [37]:
dataset = torch.utils.data.TensorDataset(X_train_ids, X_train_att, y_train) # contains all tensors for the ids, attention masks and labels
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
model = BERTClassifier1(num_classes)
train(model,initialize_weights, dataset, dataloader, 3e-5, 4)
total_pred = evaluation(model, test_dataloader, test_dataset)

Epochs: 1 | Train Loss:  0.296       | Train Accuracy:  0.573
Epochs: 2 | Train Loss:  0.079       | Train Accuracy:  0.909
Epochs: 3 | Train Loss:  0.040       | Train Accuracy:  0.967
Epochs: 4 | Train Loss:  0.045       | Train Accuracy:  0.957
Test Accuracy:  0.684


2 X (LIN LAYERS + RELU + DROPOUT = 0.2)

In [38]:
for epoch in epochs:
  print('\n')
  print('Epochs =', epoch)
  for batch in batch_sizes:
    print('\nFor a batch size of',batch)
    dataset = torch.utils.data.TensorDataset(X_train_ids, X_train_att, y_train)
    dataloader = DataLoader(dataset, batch_size=batch, shuffle=True)
    for lr in learning_rates:
      print('\n')
      model = BERTClassifier2(num_classes)
      print('Training the model')
      train(model,initialize_weights, dataset, dataloader, lr, epoch)
      total_pred = evaluation(model, test_dataloader, test_dataset)



Epochs = 3

For a batch size of 4


Training the model
Epochs: 1 | Train Loss:  0.255       | Train Accuracy:  0.635
Epochs: 2 | Train Loss:  0.041       | Train Accuracy:  0.971
Epochs: 3 | Train Loss:  0.022       | Train Accuracy:  0.981
Test Accuracy:  0.598


Training the model
Epochs: 1 | Train Loss:  0.320       | Train Accuracy:  0.511
Epochs: 2 | Train Loss:  0.065       | Train Accuracy:  0.928
Epochs: 3 | Train Loss:  0.035       | Train Accuracy:  0.963
Test Accuracy:  0.463


Training the model
Epochs: 1 | Train Loss:  0.294       | Train Accuracy:  0.542
Epochs: 2 | Train Loss:  0.118       | Train Accuracy:  0.847
Epochs: 3 | Train Loss:  0.056       | Train Accuracy:  0.942
Test Accuracy:  0.380


Training the model
Epochs: 1 | Train Loss:  0.453       | Train Accuracy:  0.221
Epochs: 2 | Train Loss:  0.249       | Train Accuracy:  0.621
Epochs: 3 | Train Loss:  0.185       | Train Accuracy:  0.734
Test Accuracy:  0.356


Training the model
Epochs: 1 | Train Loss:  0.

## Focal loss vs CE

In [39]:

dataset = torch.utils.data.TensorDataset(X_train_ids, X_train_att, y_train)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [40]:
!pip install focal_loss_torch
from focal_loss.focal_loss import FocalLoss

In [41]:

def trainFL(model, initialize_weights, gamma, train_data, data_loader, learning_rate, epochs):

    # Optional: You can specify a random seed for reproducibility
    seed_value = 42
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.apply(initialize_weights)   # initialize random weights


    loss_alg = FocalLoss(gamma=gamma)
    optimizer = Adam(model.parameters(), lr= learning_rate)
    model = model.cuda()


    for epoch in range(epochs):
      total_acc_train = 0
      total_loss_train = 0
      model.train()

      for batch in data_loader:                # batch = list of:
        input_id = batch[0].to(device)         # [batch_size, 512]
        mask = batch[1].to(device)             # [batch_size, 512]
        label = batch[2].long().to(device)     # [batch_size]

        optimizer.zero_grad()

        # FORWARD PASS

        output = model(input_id, mask) # output = tensor of [batch size, 2] with the probabilities for each of the 2 classes

        m = torch.nn.Softmax(dim=-1)

        batch_loss = loss_alg(m(output), label)   # calculate the loss between the predicted and actual label per batch
        total_loss_train += batch_loss.item()  # accumulate the loss

        acc = (output.argmax(dim=1) == label).sum().item()  # calculate the accuracy
        total_acc_train += acc

        # BACKWARD PASS
        batch_loss.backward()
        optimizer.step()        # update the gradient with the given learning rate

      print(f'Epochs: {epoch + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
      | Train Accuracy: {total_acc_train / len(train_data): .3f}')



Assessing the effect of FL vs CE on the original train data (no imbalance), 150 filtered data (high imbalance), 312 filtered data and the 2 augmented versions (lower imbalance)

In [42]:
gammas = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5)

for gamma in gammas:
  model = BERTClassifier1(num_classes)
  print('\nGamma =', gamma)
  trainFL(model,initialize_weights,gamma, dataset, dataloader, learning_rate = 3e-5, epochs = 4)
  total_pred = evaluation(model, test_dataloader, test_dataset)


Gamma = 0
Epochs: 1 | Train Loss:  0.296       | Train Accuracy:  0.573
Epochs: 2 | Train Loss:  0.073       | Train Accuracy:  0.917
Epochs: 3 | Train Loss:  0.033       | Train Accuracy:  0.965
Epochs: 4 | Train Loss:  0.056       | Train Accuracy:  0.944
Test Accuracy:  0.567

Gamma = 0.5
Epochs: 1 | Train Loss:  0.261       | Train Accuracy:  0.581
Epochs: 2 | Train Loss:  0.064       | Train Accuracy:  0.920
Epochs: 3 | Train Loss:  0.024       | Train Accuracy:  0.965
Epochs: 4 | Train Loss:  0.038       | Train Accuracy:  0.953
Test Accuracy:  0.574

Gamma = 1
Epochs: 1 | Train Loss:  0.239       | Train Accuracy:  0.598
Epochs: 2 | Train Loss:  0.058       | Train Accuracy:  0.903
Epochs: 3 | Train Loss:  0.036       | Train Accuracy:  0.936
Epochs: 4 | Train Loss:  0.016       | Train Accuracy:  0.967
Test Accuracy:  0.558

Gamma = 1.5
Epochs: 1 | Train Loss:  0.228       | Train Accuracy:  0.586
Epochs: 2 | Train Loss:  0.055       | Train Accuracy:  0.880
Epochs: 3 | Train 

In [43]:
for gamma in gammas:
  model = BERTClassifier1(num_classes)
  print('\nGamma =', gamma)
  trainFL(model,initialize_weights,gamma, aug_dataset150, aug_dataloader150, learning_rate = 3e-5, epochs = 4)
  total_pred = evaluation(model, test_dataloader, test_dataset)


Gamma = 0
Epochs: 1 | Train Loss:  0.467       | Train Accuracy:  0.354
Epochs: 2 | Train Loss:  0.275       | Train Accuracy:  0.636
Epochs: 3 | Train Loss:  0.149       | Train Accuracy:  0.809
Epochs: 4 | Train Loss:  0.198       | Train Accuracy:  0.737
Test Accuracy:  0.344

Gamma = 0.5
Epochs: 1 | Train Loss:  0.422       | Train Accuracy:  0.354
Epochs: 2 | Train Loss:  0.213       | Train Accuracy:  0.689
Epochs: 3 | Train Loss:  0.154       | Train Accuracy:  0.794
Epochs: 4 | Train Loss:  0.112       | Train Accuracy:  0.833
Test Accuracy:  0.534

Gamma = 1
Epochs: 1 | Train Loss:  0.386       | Train Accuracy:  0.373
Epochs: 2 | Train Loss:  0.166       | Train Accuracy:  0.727
Epochs: 3 | Train Loss:  0.068       | Train Accuracy:  0.890
Epochs: 4 | Train Loss:  0.073       | Train Accuracy:  0.885
Test Accuracy:  0.227

Gamma = 1.5
Epochs: 1 | Train Loss:  0.356       | Train Accuracy:  0.392
Epochs: 2 | Train Loss:  0.151       | Train Accuracy:  0.689
Epochs: 3 | Train 

In [44]:
for gamma in gammas:
  model = BERTClassifier1(num_classes)
  print('\nGamma =', gamma)
  trainFL(model,initialize_weights,gamma, aug_dataset312, aug_dataloader312, learning_rate = 3e-5, epochs = 4)
  total_pred = evaluation(model, test_dataloader, test_dataset)


Gamma = 0
Epochs: 1 | Train Loss:  0.303       | Train Accuracy:  0.539
Epochs: 2 | Train Loss:  0.104       | Train Accuracy:  0.855
Epochs: 3 | Train Loss:  0.065       | Train Accuracy:  0.927
Epochs: 4 | Train Loss:  0.031       | Train Accuracy:  0.970
Test Accuracy:  0.423

Gamma = 0.5
Epochs: 1 | Train Loss:  0.268       | Train Accuracy:  0.571
Epochs: 2 | Train Loss:  0.064       | Train Accuracy:  0.904
Epochs: 3 | Train Loss:  0.033       | Train Accuracy:  0.963
Epochs: 4 | Train Loss:  0.019       | Train Accuracy:  0.974
Test Accuracy:  0.580

Gamma = 1
Epochs: 1 | Train Loss:  0.253       | Train Accuracy:  0.541
Epochs: 2 | Train Loss:  0.050       | Train Accuracy:  0.909
Epochs: 3 | Train Loss:  0.031       | Train Accuracy:  0.958
Epochs: 4 | Train Loss:  0.033       | Train Accuracy:  0.939
Test Accuracy:  0.488

Gamma = 1.5
Epochs: 1 | Train Loss:  0.251       | Train Accuracy:  0.511
Epochs: 2 | Train Loss:  0.046       | Train Accuracy:  0.906
Epochs: 3 | Train 

# Final evaluation - Classification report

In [46]:
dataset = torch.utils.data.TensorDataset(X_train_ids, X_train_att, y_train) # contains all tensors for the ids, attention masks and labels
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
model = BERTClassifier1(num_classes)
train(model,initialize_weights, dataset, dataloader, 3e-5, 4)
total_pred = evaluation(model, test_dataloader, test_dataset)

Epochs: 1 | Train Loss:  0.296       | Train Accuracy:  0.573
Epochs: 2 | Train Loss:  0.079       | Train Accuracy:  0.909
Epochs: 3 | Train Loss:  0.040       | Train Accuracy:  0.967
Epochs: 4 | Train Loss:  0.045       | Train Accuracy:  0.957
Test Accuracy:  0.684


In [47]:
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

predicted = np.array(total_pred)    # convert to the accumulated predictions to numpy array
true_labels = np.array(y_test, dtype = int)

confusion = confusion_matrix(true_labels, predicted)
print("\nConfusion Matrix:\n", confusion)

label_names = ['Obligation', 'Omission','Permission', 'Power', 'Prohibition', 'Right']
print(classification_report(true_labels, predicted, target_names=label_names))


Confusion Matrix:
 [[71  0  0  0  0  0]
 [17  0  0  0  0  0]
 [ 0  0 66  0  0  0]
 [ 0  0  0 35  0 19]
 [ 0  0  0 17 22 19]
 [ 0  0  0 31  0 29]]
              precision    recall  f1-score   support

  Obligation       0.81      1.00      0.89        71
    Omission       0.00      0.00      0.00        17
  Permission       1.00      1.00      1.00        66
       Power       0.42      0.65      0.51        54
 Prohibition       1.00      0.38      0.55        58
       Right       0.43      0.48      0.46        60

    accuracy                           0.68       326
   macro avg       0.61      0.59      0.57       326
weighted avg       0.71      0.68      0.66       326



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
